In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D

In [2]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train / 255.0
x_test = x_test / 255.0

x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

11490434/11490434 [==============================] - 0s 0us/step


In [3]:
type(x_train[0][0, 0, 0])

numpy.float64

In [4]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(1000).batch(32)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [5]:
class MyModel(tf.keras.Model):
  def __init__(self):
    super().__init__()
    self.conv1 = Conv2D(32, 3, activation="relu")
    self.flatten = Flatten()
    self.d1 = Dense(128, activation="relu")
    self.d2 = Dense(10)

  def call(self, x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    x = self.d2(x)

    return x

model = MyModel()

In [6]:
loss_function = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.Adam()

In [7]:
train_loss = tf.keras.metrics.Mean()
test_loss = tf.keras.metrics.Mean()

train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

In [8]:
@tf.function
def train(images, labels):
  with tf.GradientTape() as tape:
    pred = model(images)
    loss = loss_function(labels, pred)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, pred)

In [9]:
@tf.function
def test(images, labels):
  pred = model(images)
  loss = loss_function(labels, pred)
  test_loss(loss)
  test_accuracy(labels, pred)

In [10]:
epochs = 5

for epoch in range(epochs):
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  #train
  for images, labels in train_dataset:
    train(images, labels)

  #test
  for images, labels in test_dataset:
    test(images, labels)

  print("Epoch: ", epoch + 1,
        f"Train loss: {train_loss.result()}",
        f"Train accuracy: {train_accuracy.result()}",
        f"Test loss: {test_loss.result()}",
        f"test accuracy: {test_accuracy.result()}")

Epoch:  1 Train loss: 0.13095970451831818 Train accuracy: 0.9605333209037781 Test loss: 0.07239152491092682 test accuracy: 0.9754999876022339
Epoch:  2 Train loss: 0.03991641104221344 Train accuracy: 0.9878666400909424 Test loss: 0.05246911197900772 test accuracy: 0.9835000038146973
Epoch:  3 Train loss: 0.01970323920249939 Train accuracy: 0.9937000274658203 Test loss: 0.05763554945588112 test accuracy: 0.982200026512146
Epoch:  4 Train loss: 0.01364370808005333 Train accuracy: 0.9955333471298218 Test loss: 0.07128847390413284 test accuracy: 0.9822999835014343
Epoch:  5 Train loss: 0.008738049305975437 Train accuracy: 0.9969000220298767 Test loss: 0.08355846256017685 test accuracy: 0.9800999760627747


In [11]:
import cv2
import numpy as np

img = cv2.imread("eight.jpg")
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img = cv2.resize(img, (28, 28))
img = img[..., tf.newaxis]
img = img[tf.newaxis, ...]
img = img.astype("float32")

# img.shape

pred = model(img)
result = np.argmax(pred)
result

8

In [12]:
model.save("/content/drive/MyDrive/models/my_mnist_model")

In [13]:
model = tf.keras.models.load_model("/content/drive/MyDrive/models/my_mnist_model")

In [18]:
import cv2
import numpy as np

img = cv2.imread("two.jpg")
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img = cv2.resize(img, (28, 28))
img = img[..., tf.newaxis]
img = img[tf.newaxis, ...]
img = img.astype("float32")

# img.shape

pred = model(img)
result = np.argmax(pred)
result

2